In [1]:
%pip install parse
%pip install GitPython

  Using cached parse-1.19.0.tar.gz (30 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24636 sha256=5a5e7b0e556497221ca63c939c9b4de49e93e159f645100c2dc6197aa7dca70e
  Stored in directory: c:\users\a_car\appdata\local\pip\cache\wheels\4c\da\8b\cb89583bfac7d6e962fca5223e5d1b7b5941861b2951760d34
Successfully built parse
Note: you may need to restart the kernel to use updated packages.
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
                                              0.0/11.2 MB ? eta -:--:--
                                              0.1/11.2 MB 3.3 MB/s eta 0:00:04
     -                                        0.5/11.2 MB 5.9 MB/s eta 0:00:02
     ----                                     1.2/11.2 MB 9.3 MB/s eta 0:00:02
     --------                                 2.5/11.2 MB 14.5 MB/s eta 0:00:01
     --------------                           4.1/11.2 MB 18.6 MB/s eta 0:00:01
     -------------------                      5.6/11.2 MB 21.0 MB/s eta 0:00:01
     ------------------------                 6.9/11.2 MB 21.9 MB/s eta 0:00:01
     -------------------------                7.0/11.2 MB 20.5 MB/s eta 0:00:01
     ----------------------------             7.9/11.2 MB 19.3 MB/s eta 0:00:01
     --------------------------------         9.0/11.2 MB 19.9 MB/s eta 0:00:01
     -----------------------------------      9.8/11.2 MB 20.2 MB

In [27]:
%pip install Office365-REST-Python-Client


                                              0.0/820.4 kB ? eta -:--:--
     --------                               184.3/820.4 kB 5.6 MB/s eta 0:00:01
     --------------------                   450.6/820.4 kB 5.6 MB/s eta 0:00:01
     -------------------------------------  809.0/820.4 kB 6.4 MB/s eta 0:00:01
     -------------------------------------- 820.4/820.4 kB 5.7 MB/s eta 0:00:00
                                              0.0/90.0 kB ? eta -:--:--
     ---------------------------------------- 90.0/90.0 kB ? eta 0:00:00
                                              0.0/2.6 MB ? eta -:--:--
     ----                                     0.3/2.6 MB 9.3 MB/s eta 0:00:01
     ------------                             0.8/2.6 MB 10.0 MB/s eta 0:00:01
     --------------------                     1.3/2.6 MB 10.5 MB/s eta 0:00:01
     ------------------------------           2.0/2.6 MB 11.5 MB/s eta 0:00:01
     ---------------------------------------  2.6/2.6 MB 11.9 MB/s eta 0:00

In [33]:
%pip install beautifulsoup4

                                              0.0/143.0 kB ? eta -:--:--
     -------------------------------------- 143.0/143.0 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import glob
import parse
import shutil
import requests
import pandas as pd
from git import Repo
from git import GitError
from io import StringIO

In [6]:
source = "https://github.com/nathanlesage/academics-on-mastodon/blob/main/README.md"


In [7]:
!wget https://raw.githubusercontent.com/nathanlesage/academics-on-mastodon/main/README.md -O 'academics-source.md'

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/carol_mb/.wget-hsts'. HSTS will be disabled.
--2023-04-13 13:15:50--  https://raw.githubusercontent.com/nathanlesage/academics-on-mastodon/main/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23525 (23K) [text/plain]
Saving to: ‘academics-source.md’

academics-source.md 100%[===================>]  22.97K  --.-KB/s    in 0.007s  

2023-04-13 13:15:51 (3.31 MB/s) - ‘academics-source.md’ saved [23525/23525]



In [3]:
file = open("academics-source.md").readlines()


In [4]:
format_string = "- [{}]({}){}"
sources = []

islist = False
for line in file:
    if line == "## Curated Account Lists\n":
        islist = True
    elif islist and line[:2] == "##":
        islist = False
    if islist and line[0] == "-":
        parsed = parse.parse(format_string, line)
        field_csv = (parsed[0], parsed[1])
        sources.append(field_csv)


In [ ]:
def get_csv(txt):
    output = []
    for line in txt.split('\n'):
        if line.startswith('@') and len(line) > 2:
            parsed = line[:-2].split(' (') #redo
            if len(parsed) > 1:
                output.append((parsed[0], parsed[1]))
    return output


def get_drive_file(source):
    field = source[0]
    url = source[1]
    str_format = "https://docs.google.com/{}/d/{}/{:z}"
    parsed = parse.parse(str_format, url, {"z": zero_or_more_string})
    doc_type = parsed[0]
    code = parsed[1]

    if doc_type == "spreadsheets":
        result = requests.get(
            "https://docs.google.com/{}/d/{}/export?format=csv".format(
                doc_type, code
            )
        )
        csv_string = result.text
        
    elif doc_type == "document":
        export_url = "https://docs.google.com/{}/d/{}/export?format=txt".format(
            doc_type, code
        )
        result = requests.get(export_url)
        csv_string = result.text
        
    return csv_string


def get_github_file(source):

    field = source[0]

    str_format = 'https://{}.github.io/{}/'
    parsed = parse.parse(str_format, source[1])
    user_git = parsed[0]
    repo_name = parsed[1]

    path_file = 'temp/' + repo_name + '_copy'
    if not os.path.isdir(path_file):
        try:
            Repo.clone_from('https://github.com/{}/{}'.format(user_git, repo_name), path_file)
        except GitError as error:
            print("Error:\n user: {}, repo: {}, msg: {}".format(user_git, repo_name, error))
            return None
    

    files = glob.glob(path_file + '/resources/*.csv')
    csv_str = ''
    for file in files:
        csv_str += open(file, encoding="utf8").read() + '\n\n'

    '''
    try:
        shutil.rmtree(path_file)
    except:
        print('permission denied to del folder temp/{}'.format(repo_name))
    
    '''
    return csv_str


def zero_or_more_string(text):
    return text


zero_or_more_string.pattern = r".*"


data = []
for source in sources:
    field = source[0].replace("/", " ")
    if "docs.google.com" in source[1]:
        txt = get_drive_file(source)
        data.append((field, txt))
    elif "sharepoint" in source[1]:
        #         https://faun.pub/quick-etl-with-python-part-1-download-files-from-sharepoint-online-40bf23711662
        pass
        
    elif "github" in source[1]:
        txt = get_github_file(source)
        if df != None:
            data.append((field, txt))
    


In [22]:
for field, field_csv in data:
    if field_csv:
        output = open('csv/scholars_{}.csv'.format(field), "w", encoding="utf-8")
        output.write(field_csv)
        output.close()

In [ ]:
https://github.com/fivestone/fedi-anthropologists/archive/refs/heads/main.zip
    
    git@github.com:fivestone/fedi-anthropologists.git